In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import itertools
from functools import total_ordering
from numpy.random import default_rng

In [ ]:
@total_ordering #allows us to only implement lt and eq, imply gt, etc.
class Disk:
    def __init__(self, x:np.ndarray, v:np.ndarray, mass:float=3, 
                 radius:float=.5, charge:float=1):
        self.m = mass
        self.r = radius
        self.x = np.asarray(x) #x example: array([2, 4, 6)
        self.v = np.asarray(v) #v example: array([1, -1, 1])
        self.nDim = len(self.x)
        if self.nDim != len(self.v):
            raise Exception("Dimensions of velocity and position lists do not match.")
        self.q = charge
    
    
    def advance(self, dt:float, L, F=0):
        # apply old velocity (update position)
        self.x += self.v * dt
        self.x = self.x % L
        
        # apply force (update velocity)
        self.v += F * (dt / self.m)
    
    #allowing comparisons between disk
    def __lt__(self, other):
        return self.x[0] < other.x[0]
    
    def __eq__(self, other):
        return self.x[0] == other.x[0]
    
    @property
    def speed(self):
        return np.linalg.norm(self.v)
    
    @property
    def KE(self):
        return (1/2)*self.m*(self.speed**2)


In [ ]:
class Expt:
    def __init__(self, particles, dt:float=0.1, t_0:float=0, 
                 tmax:float=15, L:float=20, animSpeed:float=1):
        # pPositions example: [ [1, 3], [2, 2] ]: two particles, at (1,3) and (2,2)
                
        # set time variables
        self.t = t_0
        self.tmax = tmax
        self.dt = dt
        self.animSpeed = animSpeed
        
        # make the particle list
        
        self.particles = np.asarray(particles)
        
        self.numParticles = self.particles.size
        
        self.nDim = self.particles[0].nDim 
        
        #make the box bounds
        self.L = L
    
    def nextFrame(self):
        #TODO: calculate forces
        forces = np.ones_like(self.particlePositions) #temporarily big force to check
        forceIter = iter(forces)
        for p in self.particles:
            p.advance(self.dt, self.L, next(forceIter))
        self.t += self.dt
    
    @property
    def totalKE(self):
        return sum(p.KE for p in self.particles)
    
    @property
    def avgKE(self):
        return self.totalKE / self.numParticles
    
    @property
    def particlePositions(self):
        return np.array([p.x for p in self.particles])
    
    def showAnimation(self, addlTitle=""):
        fig, ax = plt.subplots()              # create the figure
        ax.set_xlim(0,self.L)              # and adjust axes limits and labels
        ax.set_ylim(0,self.L)
        ax.set_title(self.t)
        xvar = np.linspace(0.1,self.L-0.1,self.numParticles)
        points, = ax.plot(xvar,np.ones_like(xvar), 'o')
        
        def frame(_):
            points.set_data(np.transpose(self.particlePositions))
            title = ax.set_title(addlTitle + "t = {:0.2f}".format(self.t))
            self.nextFrame()
            return points, title

        ani = FuncAnimation(fig, frame, np.arange(self.t,self.tmax,self.dt), #somewhat glitchy display. IDK how best to fix.
                            interval=self.dt*1000/self.animSpeed, blit=True)
        ani.save("particleAnimation.gif")
    
    #idea: makeCopy() function that makes an identical experiment - might be useful to
    #let us go to further times or something? idk
    

In [ ]:
seed = 42
rng = default_rng(seed=seed) # instatiate the default random number generator
sqrtNumParts = 5

pPosGrid = list(itertools.product(np.linspace(1,5,sqrtNumParts), np.linspace(1,5,sqrtNumParts)))
pVelGrid = list(rng.random(size=(sqrtNumParts**2,2))*0.01)

particles = [Disk(pPosGrid[i], pVelGrid[i]) for i in range(sqrtNumParts**2)]

e1 = Expt(particles, dt=0.5, tmax=100, animSpeed=5)

print(e1.avgKE)
e1.showAnimation("seed = " + str(seed) + "     ")
print(e1.avgKE)

In [ ]:
plt.close("all")

In [ ]:
A = np.asarray([[x, x+1] for x in range(0,50,3)])
Ai = iter(A)
print(next(Ai))
print(next(Ai))
print(next(Ai))
print(next(Ai))
print(next(Ai))
print(next(Ai))

In [ ]:
#create and test an example disk
p = Disk([1,1], [6,8], 3, 2, 1)
p1 = Disk([3, 2], [3, 3], 1, 1, 1)

print(p < p1)
print(p == p1)
print(p > p1)
print(p.speed)
print(p.KE)
print(p.x)
p.advance(2, 20)
print(p.speed)
print(p.KE)
print(p.x)
p.advance(1, 20)
print(p.speed)
print(p.KE)
print(p.x)